In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    body: str
    links: List[str]

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
caps = Website("https://www.nhl.com/capitals/")
print(caps.get_contents())

Webpage Title:
Washington Capitals | Washington Capitals
Webpage Contents:
Skip to Main Content
Tickets
All Capitals Tickets
Single Game Tickets
2024-25 Season Tickets
Partial Plans
Promotions Schedule
GOVX Military & Govt Employee Discount Program
Special Ticket Offers
Group Tickets
VIP Seating
Account Manager
Tickets for Business
Using the New NHL Mobile App
NHL Ticket Exchange
Digital Ticketing
Capital One Arena
Transformation - The Vaults
Schedule
2024-25 Season Schedule
Practice Schedule
Where To Watch
Home Jersey Schedule
Schedule Sync & Download
Team
Capitals Roster
Capitals Prospects
Caps Alumni
Management
Coaching Staff
Equipment and Trainers
Staff Directory
Monumental Sports and Entertainment
AHL Hershey Bears
ECHL South Carolina Stingrays
News
Capitals News
Capitals Today
Dump N' Chase
Community News
Ted's Take
Video
All Video
Game Highlights
Mic'd Up
Capitals Locker Room
Caps365
Rinkside Update
Off the Ice
Coach's Corner
Caps Game Entertainment
Capitals Alumni
Navigation Me

In [5]:
print(caps.links)

['#main-content', '/capitals', '/capitals', '/capitals/tickets/', 'https://www.ticketmaster.com/washington-capitals-tickets/artist/806039?brand=capitals&wt.mc_id=NHL_TEAM_WSH_SINGLE_GAME_TIX_LINK&utm_source=washcaps_tm&utm_medium=web_organic&utm_campaign=2425_sgb&utm_content=tickets_nav', 'https://www.nhl.com/capitals/club-red-365/', 'https://www.nhl.com/capitals/tickets/partial-plans', 'https://www.nhl.com/capitals/tickets/promotions', 'https://auth.govx.com/tickets/entertainer-events/39734b77-0b23-e811-80e5-14187735bc96/', 'https://www.nhl.com/capitals/tickets/offers', 'https://www.nhl.com/capitals/tickets/group-tickets', 'https://www.nhl.com/capitals/tickets/vip', 'https://am.ticketmaster.com/monumental/?wt.mc_id=NHL_TEAM_WSH_ACCOUNT_MANAGER_TIX&utm_source=washcaps_tm&utm_medium=web_organic&utm_campaign=2425sgb&utm_content=account_manager_tix', 'https://www.nhl.com/capitals/tickets/business', 'https://www.nhl.com/capitals/tickets/mobile-app-setup', 'https://www.ticketmaster.com/wash

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the team, \
such as links to an About page, Team, News, Schedule, History, Stats pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "team page", "url": "https://full.url/goes/here/team"},
        {"type": "news page": "url": "https://another.full.url/news"},
        {"type": "schedule page": "url": "https://another.full.url/schedule"},
        {"type": "history page": "url": "https://another.full.url/history"},
        {"type": "stats page": "url": "https://another.full.url/stats"},
        {"type": "standings page": "url": "https://another.full.url/standings"},
    ]
}
"""

In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the team, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, Tickets, Video, Listen, Community, Fans, Youth Hockey, Shop, League, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(caps))

Here is the list of links on the website of https://www.nhl.com/capitals/ - please decide which of these are relevant web links for a brochure about the team, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, Tickets, Video, Listen, Community, Fans, Youth Hockey, Shop, League, email links.
Links (some might be relative links):
#main-content
/capitals
/capitals
/capitals/tickets/
https://www.ticketmaster.com/washington-capitals-tickets/artist/806039?brand=capitals&wt.mc_id=NHL_TEAM_WSH_SINGLE_GAME_TIX_LINK&utm_source=washcaps_tm&utm_medium=web_organic&utm_campaign=2425_sgb&utm_content=tickets_nav
https://www.nhl.com/capitals/club-red-365/
https://www.nhl.com/capitals/tickets/partial-plans
https://www.nhl.com/capitals/tickets/promotions
https://auth.govx.com/tickets/entertainer-events/39734b77-0b23-e811-80e5-14187735bc96/
https://www.nhl.com/capitals/tickets/offers
https://www.nhl.com/capitals/tickets/group-tickets
https://www.nhl.com/capitals/tick

In [9]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://www.nhl.com/capitals/")

{'links': [{'type': 'about page', 'url': 'https://www.nhl.com/capitals/'},
  {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/management'},
  {'type': 'team page',
   'url': 'https://www.nhl.com/capitals/team/coaching-staff'},
  {'type': 'news page', 'url': 'https://www.nhl.com/capitals/news/'},
  {'type': 'schedule page', 'url': 'https://www.nhl.com/capitals/schedule'},
  {'type': 'history page', 'url': 'https://www.nhl.com/capitals/history/'},
  {'type': 'stats page', 'url': 'https://www.nhl.com/capitals/stats'},
  {'type': 'standings page', 'url': 'https://www.nhl.com/capitals/standings'}]}

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [12]:
print(get_all_details("https://www.nhl.com/capitals/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nhl.com/capitals/'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/management'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/coaching-staff'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/prospects'}, {'type': 'news page', 'url': 'https://www.nhl.com/capitals/news/'}, {'type': 'schedule page', 'url': 'https://www.nhl.com/capitals/schedule'}, {'type': 'history page', 'url': 'https://www.nhl.com/capitals/history/'}, {'type': 'stats page', 'url': 'https://www.nhl.com/capitals/stats'}, {'type': 'standings page', 'url': 'https://www.nhl.com/capitals/standings'}]}
Landing page:
Webpage Title:
Washington Capitals | Washington Capitals
Webpage Contents:
Skip to Main Content
Tickets
All Capitals Tickets
Single Game Tickets
2024-25 Season Tickets
Partial Plans
Promotions Schedule
GOVX Military & Govt Employee Discount Program
Special Ticket Offers
Group Tickets
VIP Seating
Accou

In [13]:
system_prompt = "You are a sports marketing analyst that analyzes the contents of several relevant pages from a sports team website \
and creates a short brochure about the team for prospective fans and players to recruit. Respond in markdown.\
Include details of team history, team culture, team news, and team stats if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the team in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:40_000] # Truncate if more than 40,000 characters
    return user_prompt

In [15]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [16]:
create_brochure("Washington Capitals", "https://www.nhl.com/capitals")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nhl.com/capitals'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/management'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/coaching-staff'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/equipment-training-staff'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/staff'}, {'type': 'news page', 'url': 'https://www.nhl.com/capitals/news/'}, {'type': 'schedule page', 'url': 'https://www.nhl.com/capitals/schedule'}, {'type': 'history page', 'url': 'https://www.nhl.com/capitals/history/'}, {'type': 'history page', 'url': 'https://www.nhl.com/capitals/history/50th-anniversary'}, {'type': 'stats page', 'url': 'https://www.nhl.com/capitals/stats'}, {'type': 'standings page', 'url': 'https://www.nhl.com/capitals/standings'}, {'type': 'roster page', 'url': 'https://www.nhl.com/capitals/roster'}, {'type': 'prospects page', 'url': 'https://www.nhl.com/capitals/

# Washington Capitals Team Brochure

---

## 🏒 Welcome to the Washington Capitals!

### Team History
Founded in 1974, the Washington Capitals have become a cornerstone of professional ice hockey in the NHL. Over the decades, they've shone brightly in the league, culminating in their first Stanley Cup victory in 2018. This remarkable achievement marked the pinnacle of a journey that began with countless playoff appearances and franchise milestones.

### Celebrating 50 Years
Join us in celebrating the **50th Anniversary** of Capitals hockey! Reminisce about the highs and lows that shaped this legendary team and the passionate fanbase that stands behind them.

### Notable Achievements
- **2018 Stanley Cup Champions**
- Three Presidents' Trophy winners (2009-10, 2015-16, 2016-17)
- A record of 1,616-1,155-201-185 since 1982 

### Team Culture
The Capitals embody a culture of excellence, teamwork, and community involvement. Guided by the motto **"A Legacy in ALL CAPS,"** our players not only compete on the ice but also actively engage in community initiatives, promoting diversity and inclusion through programs like **Caps in the Community**.

### Coaching Staff
Led by **Spencer Carbery**, named head coach in 2023, our experienced coaching staff—composed of former players and successful coaches—works tirelessly to develop our talent and tactical strategies. 

### Team Statistics (As of October 2023)
- Total Games Played: Over 3,000
- Career Points Leader: Alex Ovechkin
- Regular Season Achievements: 
  - Most Wins in a Decade (2010s)
  - Record of 56 wins in 2015-16 season

### Community Engagement
The Capitals are committed to giving back, whether it's through youth programs, military initiatives, or local partnerships. Our community-focused initiatives touch lives and promote the love of hockey among the younger generations.

### Team News & Updates
Stay updated with the latest happenings regarding trades, player signings, and game highlights through our news portal. Follow acclaimed segments like **“Dump N' Chase”** for insightful coverage of the team's journey.

### Join Us!
Whether you're a lifelong fan or a newcomer, we invite you to become part of the Capitals family. Support our players by attending games at **Capital One Arena**, enjoying the electric atmosphere that only live hockey can offer.

### Tickets & Memberships
Explore various ticketing options from single-game passes to season-long commitments. Visit our **[Official Website](https://www.washingtoncaps.com)** for more details on tickets and joining the **Caps Kids Club** for our younger fans!

---

## 🌟 Get Involved
**Connect with us on Social Media:**
- [Facebook](https://www.facebook.com/washingtoncapitals)
- [Twitter](https://twitter.com/Capitals)
- [Instagram](https://www.instagram.com/capitals)

Be part of an unforgettable experience with the Washington Capitals—where passion meets performance on and off the ice!

# Translate to Spanish

In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure in spanish of the team in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:40_000] # Truncate if more than 40,000 characters
    return user_prompt

In [19]:
create_brochure("Washington Capitals", "https://www.nhl.com/capitals")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nhl.com/capitals'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/management'}, {'type': 'team page', 'url': 'https://www.nhl.com/capitals/team/coaching-staff'}, {'type': 'schedule page', 'url': 'https://www.nhl.com/capitals/schedule'}, {'type': 'news page', 'url': 'https://www.nhl.com/capitals/news'}, {'type': 'history page', 'url': 'https://www.nhl.com/capitals/history'}, {'type': 'stats page', 'url': 'https://www.nhl.com/capitals/stats'}, {'type': 'standings page', 'url': 'https://www.nhl.com/capitals/standings'}, {'type': 'roster page', 'url': 'https://www.nhl.com/capitals/roster'}, {'type': 'prospects page', 'url': 'https://www.nhl.com/capitals/prospects'}]}


# Washington Capitals

## Bienvenidos a los Capitals de Washington

### Historia del Equipo
Los Washington Capitals han sido una parte integral de la historia del hockey sobre hielo desde su fundación en 1974. Durante sus 50 años, han enfrentado desafíos y han logrado éxitos notables, incluyendo su primera victoria en la Copa Stanley en 2018, marcando un hito en la historia del equipo. Los Capitals son conocidos por su fuerte espíritu competitivo y su dedicación a la excelencia en el deporte.

### Cultura del Equipo
La cultura de los Capitals se basa en la inclusión, el trabajo en equipo y una fuerte conexión con la comunidad. A través de programas de diversidad, equidad e inclusión, los Capitals no solo buscan ser un gran equipo en el hielo, sino también un pilar en su comunidad. El equipo participa activamente en la comunidad a través de diversas iniciativas y programas que fomentan el amor por el hockey entre los jóvenes.

### Noticias del Equipo
Para mantenerte al tanto de las novedades, puedes seguir las actualizaciones diarias y los análisis en nuestro sitio web. Desde reseñas de partidos hasta entrevistas con jugadores y el cuerpo técnico, siempre hay algo nuevo que descubrir sobre tus Capitals.

### Estadísticas Destacadas
- **Victorias en la NHL:** Desde 1982, los Capitals han sido una fuerza dominante en la NHL, acumulando más de 1,600 victorias.
- **Mejor Década:** Durante la última década, los Capitals tuvieron el récord de más victorias en la NHL, destacándose como uno de los equipos más exitosos.
- **Copa Stanley:** Campeones de la Copa Stanley en 2018, con un impresionante récord de victorias en los playoffs.

### ¿Por qué unirse a los Capitals?
Invitamos a todos los fanáticos, nuevos o actuales, a unirse a nosotros en la Capital One Arena para vivir la pasión del hockey. Con diversos planes de boletos y eventos especiales, siempre hay una manera de disfrutar de un juego de hockey emocionante. Además, los aficionadores pueden formar parte de clubes exclusivos y disfrutar de ventajas especiales.

### Redes Sociales
¡Síguenos en nuestras plataformas sociales para no perderte nada de la acción!
- [Facebook](https://www.facebook.com/washingtoncapitals)
- [Instagram](https://www.instagram.com/washingtoncapitals)
- [Twitter](https://twitter.com/capitals)

---

**¡Ven y únete a la familia de los Washington Capitals, donde cada juego es una celebración!** 🎉